In [1]:
import pandas as pd
import sqlite3
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, DateTime, insert

# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 160)

In [2]:
# Connect to database
engine = create_engine(f"mysql+mysqlconnector://{name}:{password}@{host}:3306/{db}", echo=True)
meta = MetaData()

# Create table variable - link to table in database
stop_proportions = Table(
   'stop_proportions', meta, 
    Column("LINEID", String(12)),
    Column("ROUTEID", String(12)),
    Column("DIRECTION", Integer),
    Column("PROGRNUMBER", Integer),
    Column("STOPPOINTID", Integer),
    Column("STOP_PERCENT", Float) 
)

In [3]:
# Read in original trips dataframe (all lines included)
trips = pd.read_csv("rt_trips_DB_2018.txt", sep=";")

In [4]:
trips.head()

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [5]:
trips.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
BASIN               object
TENDERLOT          float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
dtype: object

In [6]:
# Cast LINEID to type string, to ensure all entries are of type string
trips["LINEID"] = trips["LINEID"].astype(str)

In [7]:
# Drop unnecessary features 
trips.drop(["DATASOURCE", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP", "BASIN", "TENDERLOT", "SUPPRESSED", "JUSTIFICATIONID", "LASTUPDATE", "NOTE"], axis = 1, inplace=True)

In [8]:
trips.isna().sum()

DAYOFSERVICE    0
TRIPID          0
LINEID          0
ROUTEID         0
DIRECTION       0
dtype: int64

In [9]:
trips.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION
0,07-FEB-18 00:00:00,6253783,68,68_80,1
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1
4,07-FEB-18 00:00:00,6253175,14,14_15,1


In [10]:
# Read in list of most common subroutes
with open("most_common_subroutes.txt") as file:
    most_common_subroutes = file.readlines()
most_common_subroutes = [line.strip() for line in most_common_subroutes] 

In [11]:
print(most_common_subroutes)

['102_8', '102_9', '104_15', '104_16', '111_7', '111_8', '114_5', '114_6', '116_1', '116_3', '118_4', '11_40', '11_42', '120_7', '120_9', '122_18', '122_20', '123_34', '123_36', '130_10', '130_11', '13_60', '13_67', '140_19', '140_21', '142_12', '142_13', '145_102', '145_105', '14C_17', '14C_18', '14_15', '14_16', '150_8', '150_9', '151_15', '151_17', '15A_83', '15A_84', '15B_60', '15B_61', '15D_62', '15D_63', '15_16', '15_17', '161_50', '161_51', '16C_28', '16C_29', '16D_30', '16_20', '16_24', '17A_15', '17A_17', '17_10', '17_15', '184_28', '184_29', '185_53', '185_56', '18_3', '18_4', '1_37', '1_40', '220_10', '220_12', '236_10', '236_9', '238_11', '238_15', '239_26', '239_28', '25A_270', '25A_273', '25B_271', '25B_274', '25D_275', '25D_277', '25X_10', '25X_11', '25_269', '25_272', '26_28', '26_29', '270_42', '270_44', '27A_4', '27A_5', '27B_23', '27B_34', '27X_42', '27X_43', '27_17', '27_19', '29A_14', '29A_15', '31A_25', '31A_26', '31B_44', '31B_46', '31D_50', '31D_51', '31_15', '3

In [12]:
problem_stops_df = pd.DataFrame(columns=["LINEID", "DIRECTION", "STOPPOINTID"])

In [13]:
# Create dataframe of stops per line not included on most common subroutes
for route in most_common_subroutes:
    line = route.split("_")[0]
    
    line_df = pd.read_csv("stoptimes/" + line + "_stops.csv")
    merged = line_df.merge(trips, how="left", on=["DAYOFSERVICE", "TRIPID"])
    merged.dropna(inplace=True)
    
    route_subset = merged.loc[merged["ROUTEID"] == route]
    stops_per_route = route_subset.STOPPOINTID.unique().tolist()

    direction = route_subset.iloc[0]["DIRECTION"]
    direction_subset = merged.loc[merged["DIRECTION"] == direction]
    stops_per_direction = direction_subset.STOPPOINTID.unique().tolist()
    
    extra_stops = np.setdiff1d(stops_per_direction, stops_per_route).tolist()
    
    if extra_stops == []:
        print(route, "ok")
    else:
        for stop in extra_stops:
            new_row_problem_stops_df = {"LINEID":line, "DIRECTION":direction, "STOPPOINTID":stop}
            problem_stops_df = problem_stops_df.append(new_row_problem_stops_df, ignore_index=True)
        print("Stops added for route", route)

102_8 ok
102_9 ok
104_15 ok
104_16 ok
111_7 ok
Stops added for route 111_8
114_5 ok
114_6 ok
116_1 ok
116_3 ok
Stops added for route 118_4
11_40 ok
11_42 ok
Stops added for route 120_7
Stops added for route 120_9
122_18 ok
122_20 ok
123_34 ok
123_36 ok
130_10 ok
130_11 ok
Stops added for route 13_60
Stops added for route 13_67
140_19 ok
140_21 ok
142_12 ok
Stops added for route 142_13
Stops added for route 145_102
Stops added for route 145_105
14C_17 ok
14C_18 ok
14_15 ok
Stops added for route 14_16
150_8 ok
150_9 ok
151_15 ok
151_17 ok
15A_83 ok
Stops added for route 15A_84
15B_60 ok
Stops added for route 15B_61
15D_62 ok
Stops added for route 15D_63
15_16 ok
15_17 ok
161_50 ok
161_51 ok
16C_28 ok
Stops added for route 16C_29
16D_30 ok
Stops added for route 16_20
16_24 ok
17A_15 ok
17A_17 ok
Stops added for route 17_10
17_15 ok
184_28 ok
184_29 ok
Stops added for route 185_53
Stops added for route 185_56
18_3 ok
18_4 ok
1_37 ok
1_40 ok
Stops added for route 220_10
Stops added for rout

In [14]:
# Store problem_stops_df as csv file for later access
problem_stops_df.to_csv("stops_not_on_most_common.csv", index=False)

In [15]:
# Read in dataframe of problem stops
problem_stops_df = pd.read_csv("stops_not_on_most_common.csv")

In [16]:
problem_stops_df.head()

,LINEID,DIRECTION,STOPPOINTID
0,111,2,7648
1,118,2,334
2,118,2,345
3,120,1,279
4,120,1,400


In [17]:
# 584 stops aren't included on the most common subroutes and will need to be dealt with 
problem_stops_df.shape

(584, 3)

In [18]:
# Cast LINEID to type string, to ensure all entries are of type string
problem_stops_df["LINEID"] = problem_stops_df["LINEID"].astype(str)

In [19]:
# Create dataframe of problem stops on OB lines
ob_problem_stops_df = problem_stops_df.loc[problem_stops_df["DIRECTION"] == 1]

In [20]:
ob_problem_stops_df.head()

,LINEID,DIRECTION,STOPPOINTID
3,120,1,279
4,120,1,400
5,120,1,485
6,120,1,487
7,120,1,488


In [21]:
# Create dataframe of problem stops on IB lines
ib_problem_stops_df = problem_stops_df.loc[problem_stops_df["DIRECTION"] == 2]

In [22]:
ib_problem_stops_df.head()

,LINEID,DIRECTION,STOPPOINTID
0,111,2,7648
1,118,2,334
2,118,2,345
15,120,2,273
16,120,2,405


<h4>Lines with problem stops</h4>

In [26]:
# Create list of outbound lines with problematic stops
ob_problematic = problem_stops_df.loc[problem_stops_df["DIRECTION"] == 1]
problematic_lines_ob = ob_problematic.LINEID.unique().tolist()

In [27]:
print(problematic_lines_ob)

['120', '13', '142', '145', '16C', '16', '17', '185', '220', '27B', '27', '31', '32X', '33', '38', '39', '40B', '40D', '40', '41B', '41X', '41', '43', '45A', '46A', '47', '53', '61', '63', '65', '66X', '67X', '68', '75', '7A', '7', '83A', '83', '84A', '84X', '84']


In [28]:
# Create list of inbound lines with problematic stops
ib_problematic = problem_stops_df.loc[problem_stops_df["DIRECTION"] == 2]
problematic_lines_ib = ib_problematic.LINEID.unique().tolist()

In [29]:
print(problematic_lines_ib)

['111', '118', '120', '13', '145', '14', '15A', '15B', '15D', '185', '220', '25X', '27A', '27B', '27', '33A', '33X', '33', '38A', '38', '39X', '40D', '41X', '41', '43', '45A', '46A', '47', '49', '51D', '51X', '61', '63', '65', '66X', '67X', '68', '75', '7D', '83A', '83', '84A', '84X', '84', '9']


In [30]:
# Read in dataframe of stops per most common subroutes
stops_most_common = pd.read_csv("stops_per_most_common_subroutes.csv")

In [31]:
stops_most_common.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID
0,102,102_8,1,1,4381
1,102,102_8,1,2,935
2,102,102_8,1,3,936
3,102,102_8,1,4,913
4,102,102_8,1,5,914


<h4>Calculate proportions</h4>

In [32]:
# Trips dataframe holding only data for main subroutes
main_trips = pd.read_csv("trips_to_match_leavetimes.csv")

In [33]:
main_trips.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JOURNEYTIME
0,07-FEB-18 00:00:00,6253917,102,102_8,1,87253,84600,86999,84588,2411
1,07-FEB-18 00:00:00,6250433,102,102_8,1,29588,26100,30330,26175,4155
2,07-FEB-18 00:00:00,6253905,102,102_8,1,31737,28200,32979,28292,4687
3,07-FEB-18 00:00:00,6260556,102,102_8,1,33237,29700,33432,29770,3662
4,07-FEB-18 00:00:00,6259508,102,102_8,1,34437,30900,34743,30893,3850


In [34]:
main_trips.isna().sum()

DAYOFSERVICE       0
TRIPID             0
LINEID             0
ROUTEID            0
DIRECTION          0
PLANNEDTIME_ARR    0
PLANNEDTIME_DEP    0
ACTUALTIME_ARR     0
ACTUALTIME_DEP     0
JOURNEYTIME        0
dtype: int64

In [35]:
# Cast LINEID to type string, to ensure all entries are of type string
main_trips["LINEID"] = main_trips["LINEID"].astype(str)

In [36]:
main_trips.dtypes

DAYOFSERVICE       object
TRIPID              int64
LINEID             object
ROUTEID            object
DIRECTION           int64
PLANNEDTIME_ARR     int64
PLANNEDTIME_DEP     int64
ACTUALTIME_ARR      int64
ACTUALTIME_DEP      int64
JOURNEYTIME         int64
dtype: object

In [ ]:
def match_first_on_main_subroute(list_lines, prob_stops_df, direction):
    still_problem = {}

    for line in list_lines:
        print("LINE", line)

    #   Read in dataframe of all stops for current line
        line_df = pd.read_csv("stoptimes/" + line + "_stops.csv")
        merged = line_df.merge(trips, how="left", on=["DAYOFSERVICE", "TRIPID"])
        merged.dropna(inplace=True)
        merged.drop(["PLANNEDDEP_STOP", "PLANNEDARR_STOP"], axis = 1, inplace=True)

    #   Get list of problematic stops for current line
        current_prob_stops_df = prob_stops_df.loc[prob_stops_df["LINEID"] == line]
        prob_stops_list = current_prob_stops_df.STOPPOINTID.unique().tolist()

    #   Get main subroute starting stop for current line
        first_stop_df = stops_most_common.loc[(stops_most_common["LINEID"] == line) & (stops_most_common["DIRECTION"] == direction)]
        first_stop = first_stop_df.loc[first_stop_df["PROGRNUMBER"] == first_stop_df["PROGRNUMBER"].min()].iloc[0]["STOPPOINTID"]
        print("Target stop", first_stop)

    #   Get list of route options for current line
        routes = merged.ROUTEID.unique().tolist()

        still_prob = []

        for stop in prob_stops_list:
            current_list = [stop, first_stop]
            print("Stop", stop)

            for route in routes:
                current_route = merged.loc[merged["ROUTEID"] == route]
                current_route_stops = current_route.STOPPOINTID.unique().tolist()
                current_route_max_progr = current_route.PROGRNUMBER.max()
                print(current_route_max_progr)

                check_stops = all(item in current_route_stops for item in current_list)

                if check_stops:
                    print("Route" , route , "for stop", stop)
                    stop_df = current_route.loc[current_route["STOPPOINTID"] == stop]

                    target_stop_df = current_route.loc[current_route["STOPPOINTID"] == first_stop]
                    new_merge = stop_df.merge(target_stop_df, how="left", on=["TRIPID"], suffixes=("_prob", "_first"))
                    new_merge["DIST_IN_SECS"] = new_merge["ACTUALARR_STOP_prob"] - new_merge["ACTUALDEP_STOP_first"]
                    median_dist_in_secs = new_merge["DIST_IN_SECS"].median()
                    median_trip_journeytime = main_trips.loc[(main_trips["LINEID"] == line) & (main_trips["DIRECTION"] == direction)].JOURNEYTIME.median()
                    stop_percent = median_dist_in_secs / median_trip_journeytime * 100
                    print("STOP %", stop_percent)

                    if stop_percent < 0:
                        progrnum = -current_route_max_progr + stop_df.iloc[0]["PROGRNUMBER"]
                    else:
                        progrnum = stop_df.iloc[0]["PROGRNUMBER"]

                    print(progrnum)

    #               Insert new row into database table stop_proportions
                    ins = stop_proportions.insert().values(LINEID=line, ROUTEID=route, DIRECTION=0, PROGRNUMBER=progrnum, STOPPOINTID=stop, STOP_PERCENT=stop_percent)
                    conn = engine.connect()
                    result = conn.execute(ins)

                    break

            else:
                still_prob.append(stop)

        if still_prob != []:
            still_problem[line] = still_prob
        
        return still_problem

In [41]:
still_prob_ob = match_to_first_on_main_subroute(problematic_lines_ob, ob_prob_stops_df, 1)

LINE 120
Target stop 6004
Stop 279
26
24
34
37
34
24
26
Stop 400
26
24
34
37
34
24
26
Stop 485
26
24
34
37
34
24
26
Stop 487
26
24
34
37
34
24
26
Stop 488
26
24
34
37
34
24
26
Stop 489
26
24
34
37
34
24
26
Stop 490
26
24
34
37
34
24
26
Stop 491
26
24
34
37
34
24
26
Stop 492
26
24
34
37
34
24
26
Stop 493
26
24
34
37
34
24
26
Stop 494
26
24
34
37
34
24
26
Stop 495
26
24
34
37
34
24
26
LINE 13


KeyboardInterrupt: 

In [ ]:
print(still_problem_ob)

In [ ]:
# Hardcoding still_prob_outbound, to return to later
still_problem_ob = {"120": [279, 400, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495], "145": [767, 6000], "16C": [208, 209, 210, 211, 212, 215, 216, 217, 218, 219, 1641, 1642], "16": [208, 209, 210, 211, 212, 226, 227, 228, 229, 230, 1641, 1642], "27": [395, 396, 397, 398, 399, 400], "39": [767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 7497], "40B": [4592], "46A": [4962], "53": [271, 286, 513, 1172, 4380, 4513, 7671], "7A": [1174, 3042], "7": [1174], "83A": [7132], "83": [7132], "84A": [422, 423, 424, 425, 426, 427, 429, 3032], "84X": [4952]}

In [ ]:
still_problem_ib = match_to_first_on_main_subroute(problematic_lines_ib, ib_prob_stops_df, 2)

In [ ]:
print(still_problem_ib)

In [ ]:
still_problem_ib = {"13": [7230], "145": [4533, 4844], "14": [254], "27A": [956, 957, 958, 959, 960, 961], "27B": [1176, 1177, 1178, 1179], "41X": [3686, 3694, 3695, 3751, 3752, 3884, 4843, 4905, 4906, 4907, 4908, 4909, 4921, 4922, 4923, 4924, 4958, 5077, 7114, 7116, 7407, 7654, 7655], "41": [5044], "43": [4923, 4924, 4958], "49": [2353, 2357, 2359], "65": [7208, 7249, 7251, 7253, 7255, 7256, 7259, 7261, 7262, 7265, 7266, 7269, 7270, 7280, 7281, 7284, 7287, 7289], "66X": [3994, 3995, 3996, 4003, 4404, 4405], "68": [1471, 1472, 1473, 1990, 1992, 4408], "84X": [4260, 4266, 7296]}

In [ ]:
def match_last_on_main_subroute(stops_dict, direction)
    still_problem = {}

    for line, stops in stops_dict.items():
        print("LINE", line)

        line_df = pd.read_csv("stoptimes/" + line + "_stops.csv")
        merged = line_df.merge(trips, how="left", on=["DAYOFSERVICE", "TRIPID"])
        merged.dropna(inplace=True)
        merged.drop(["PLANNEDDEP_STOP", "PLANNEDARR_STOP"], axis = 1, inplace=True)

        last_stop_df = stops_most_common.loc[(stops_most_common["LINEID"] == line) & (stops_most_common["DIRECTION"] == direction)]
        last_stop = last_stop_df.loc[last_stop_df["PROGRNUMBER"] == last_stop_df["PROGRNUMBER"].max()].iloc[0]["STOPPOINTID"]
        print("Target stop", last_stop)

        routes = merged.ROUTEID.unique().tolist()

        still_prob = []

        for stop in stops:
            current_list = [stop, last_stop]
            print("Stop", stop)

            for route in routes:
                current_route = merged.loc[merged["ROUTEID"] == route]
                current_route_stops = current_route.STOPPOINTID.unique().tolist()

                check_stops = all(item in current_route_stops for item in current_list)

                if check_stops:
                    print("Route" , route , "for stop", stop)
                    stop_df = current_route.loc[current_route["STOPPOINTID"] == stop]
                    target_stop_df = current_route.loc[current_route["STOPPOINTID"] == last_stop]

                    new_merge = stop_df.merge(target_stop_df, how="left", on=["DAYOFSERVICE", "TRIPID"], suffixes=("_prob", "_last"))
                    new_merge["DIST_IN_SECS"] = new_merge["ACTUALARR_STOP_last"] - new_merge["ACTUALDEP_STOP_prob"] 
                    median_dist_in_secs = new_merge["DIST_IN_SECS"].median()
                    median_trip_journeytime = main_trips.loc[(main_trips["LINEID"] == line) & (main_trips["DIRECTION"] == direction)].JOURNEYTIME.median()
                    stop_percent = 100 - (median_dist_in_secs / median_trip_journeytime * 100)
                    print("STOP %", stop_percent)
                    
                    if stop_percent > 0:
                        progrnum = current_route_max_progr + stop_df.iloc[0]["PROGRNUMBER"]
                    else:
                        progrnum = stop_df.iloc[0]["PROGRNUMBER"]

                    ins = stop_proportions.insert().values(LINEID=line, DIRECTION=direction-1, PROGRNUMBER=progrnum, STOPPOINTID=stop, STOP_PERCENT=stop_percent)
                    conn = engine.connect()
                    result = conn.execute(ins)

                    break

            else:
                still_prob.append(stop)
        if still_prob != []:
            still_problem_ob_2[line] = still_prob


In [ ]:
still_problem_ob_2 = match_last_on_main_subroute(still_prob_ob, 1)

In [ ]:
print(still_problem_ob_2)

In [ ]:
# Hardcoding still_problem_ob_2 to return to later
still_problem_ob_2 = {"145": [767, 6000], "39": [7497]}

In [ ]:
still_problem_ib_2 = match_last_on_main_subroute(still_prob_ib, 2)

In [ ]:
print(still_problem_ib_2)

In [ ]:
still_problem_ib_2 = {"41X": [7655], "68": [1471, 1472, 1473, 1990, 1992, 4408]}

<p>The above stops remain; 3 for outbound journeys and 7 for inbound journeys.</p>